# Python notebook for ad hoc testing.






In [ ]:
%matplotlib widget
import glob
import ipywidgets as widgets

# global variables
results_sel = ""  # the selected results directory

# create results directory widget
result_dirs = sorted([f.split('/')[-2] for f in glob.glob("../results/*/", recursive=False)], key=str.casefold)
results_widget = widgets.Select(options=result_dirs, description='Results dir', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# display and respond to the widgets
def f(w1):
  global results_sel
  results_sel = results_widget.value
display(widgets.interactive(f, w1=results_widget))


#### Peak counting over selected time range.
Use the interactive sliders to set the start and finish times.

In [ ]:
#global variables
istart = 0
idone = 0
idx = []

with open("../results/" + data_file, 'r', encoding='utf-8-sig') as f: 
  A0 = np.transpose(np.genfromtxt(f, dtype=float, delimiter=','))
wtmax = np.ceil(A0[0,-1])
dmin = np.amin(A0[data_col])
dmax = np.amax(A0[data_col])
count_start = 0.0   # count start time
count_done = wtmax  # count end time
p = 4000            # desired length of resampled data

X0 = A0[0]
Y0 = A0[data_col]
tmin = np.min(X0)            # start time
tmax = np.max(X0)            # finish time
Y1 = (Y0-dmin) / (dmax-dmin) # data axis, normalized to range(0, 1.0)
sr = p / (tmax-tmin)         # the sample rate

# resample
f = interp1d(X0, Y1, kind='cubic')                    # define the resampling function
X = np.linspace(tmin, tmax, p+1, endpoint=True)       # define the new time steps
Y = f(X)                                              # resample the original signal

# apply high-pass filter to eliminate the stimulation "bump" in the data
sos = signal.butter(3, 0.1, btype='highpass', fs=sr, output='sos')
Yf = signal.sosfiltfilt(sos, Y) # zero phase shift filter

# apply low-pass filter to smooth out higher frequencies in the data
sos = signal.butter(7, 2.0, btype='lowpass', fs=sr, output='sos')
Yf = signal.sosfiltfilt(sos, Yf) # zero phase shift filter

pks,_ = signal.find_peaks(Yf,prominence=0.04)       # find indices of peaks in the resampled, filtered data
pidx = np.around((X0.shape[0]-1)*pks/p).astype(int) # convert to indices in the original data
pts = [A0[0][pidx], A0[data_col][pidx]]             # save the peaks as points in the original data

plt.close('all')
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(12,6), constrained_layout=True)

@interact(
  cs = widgets.FloatSlider(value=count_start, min=0.0, max=wtmax-1.0, step=1.0,
                    description='start time', layout=Layout(width='700px')),
  cd = widgets.FloatSlider(value=count_done, min=1.0, max=wtmax, step=1.0,
                    description='finish time', layout=Layout(width='700px'))
)

def f(cs,cd):
  global istart, idone, idx
  istart = np.where(A0[0] > cs)[0][0]
  idone = np.where(A0[0] < cd)[0][-1]

  ax[0].cla()
  ax[0].set_title("Time range selection")
  ax[0].set(ylabel="intensity")
  ax[0].set(xlabel="time (s)")
  ax[0].plot(A0[0],A0[data_col])
  ax[0].plot([cs,cs],[dmin,dmax],color='green')
  ax[0].plot([cd,cd],[dmin,dmax],color='red')

  ax[1].cla()
  ax[1].set_title("Peaks in time range")
  ax[1].set(ylabel="intensity")
  ax[1].set(xlabel="time (s)")

  idx = pidx[(pidx >= istart) & (pidx <= idone)]
  ax[1].plot(A0[0,istart:idone],A0[data_col,istart:idone])
  ax[1].plot(A0[0,idx],A0[data_col,idx],'k.')                                # plot the peak locations
    
  plt.show()
  return("peaks count: " + str(len(idx)))

In [ ]:
import csv

# save peaks to CSV file
with open ("../results/peaks.csv", 'w') as file:
  writer = csv.writer(file)
  for row in np.transpose(A0[:,idx]):
    writer.writerow([row[0], row[data_col]])
